# Day 11: Introduction to Value Investing

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/astoreyai/money-talks/blob/main/class3_trading_strategies/week3_value_growth/day11_intro_value_investing.ipynb)

---

## Learning Objectives

1. **Understand** the principles of value investing
2. **Learn** the philosophy of Benjamin Graham and Warren Buffett
3. **Identify** undervalued stocks using basic metrics
4. **Apply** margin of safety concept
5. **Distinguish** value investing from trading

---

## Lecture (30 minutes)

### What is Value Investing?

**Value investing** is an investment strategy focused on buying stocks that appear underpriced relative to their intrinsic value.

```
VALUE INVESTING PHILOSOPHY
==========================

CORE PRINCIPLE:
"Price is what you pay, value is what you get."
                                - Warren Buffett

THE CONCEPT:

  Market Price -----> What stock trades at
       vs.
  Intrinsic Value --> What company is actually worth

VALUE OPPORTUNITY:
When Price < Intrinsic Value = BUY
When Price > Intrinsic Value = SELL/AVOID

KEY ASSUMPTION:
Markets are inefficient in the short term,
but prices converge to value over time.
```

### The Founders

```
VALUE INVESTING LEGENDS
=======================

BENJAMIN GRAHAM (1894-1976)
"Father of Value Investing"
- Authored "The Intelligent Investor"
- Developed margin of safety concept
- Taught Warren Buffett at Columbia
- Focused on quantitative analysis
- Net-net working capital strategy

WARREN BUFFETT (1930-present)
"Oracle of Omaha"
- Graham's most famous student
- Evolved value investing with quality focus
- "Wonderful company at fair price"
- Long-term holding philosophy
- Berkshire Hathaway: 20%+ annual returns

CHARLIE MUNGER (1924-2023)
- Buffett's partner
- Quality over pure value
- Multi-disciplinary mental models
```

### Value Investing vs Trading

```
INVESTING vs TRADING
====================

                    VALUE INVESTING      TRADING
                    ---------------      -------
Time Horizon        Years to decades     Minutes to weeks
Focus               Business quality     Price movement
Analysis            Fundamentals         Technical
Frequency           Few trades/year      Many trades/month
Effort              Deep research        Chart reading
Edge                Business analysis    Timing/momentum
Taxes               Long-term gains      Short-term gains

VALUE INVESTOR MINDSET:
- You're buying a business, not a ticker
- Price drops = opportunity, not problem
- Patience is competitive advantage
- Ignore daily price fluctuations
```

### Margin of Safety

```
MARGIN OF SAFETY
================

The central concept of value investing:

Margin of Safety = Intrinsic Value - Purchase Price
                   --------------------------------
                        Intrinsic Value

EXAMPLE:
Intrinsic Value: $100
Purchase Price:  $70
Margin of Safety: 30%

WHY IT MATTERS:
1. Protects against estimation errors
2. Accounts for unforeseen problems
3. Limits downside risk
4. Increases potential returns

GRAHAM'S GUIDELINES:
- Minimum 25-33% margin of safety
- Higher for riskier companies
- Never pay full value
```

### Key Value Metrics

```
FUNDAMENTAL VALUE RATIOS
========================

P/E RATIO (Price to Earnings):
P/E = Stock Price / Earnings Per Share
- Low P/E = potentially undervalued
- Compare to industry average
- Graham's rule: P/E < 15

P/B RATIO (Price to Book):
P/B = Stock Price / Book Value Per Share
- P/B < 1 = trading below asset value
- Graham's rule: P/B < 1.5

GRAHAM NUMBER:
= sqrt(22.5 × EPS × Book Value)
Maximum price to pay per Graham

P/E × P/B < 22.5 (Graham's combined rule)

DEBT/EQUITY:
= Total Debt / Shareholders Equity
- Lower is safer
- Graham's rule: D/E < 1.0

CURRENT RATIO:
= Current Assets / Current Liabilities
- Measures short-term solvency
- Graham's rule: > 2.0
```

---

## Hands-On Practice (15 minutes)

In [ ]:
!pip install yfinance pandas numpy matplotlib -q
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
print("Libraries loaded!")

In [ ]:
def get_fundamentals(ticker):
    """Get key fundamental data for a stock."""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        
        data = {
            'Ticker': ticker,
            'Name': info.get('shortName', 'N/A'),
            'Sector': info.get('sector', 'N/A'),
            'Price': info.get('currentPrice', info.get('regularMarketPrice', 0)),
            'Market Cap': info.get('marketCap', 0),
            'PE Ratio': info.get('trailingPE', 0),
            'Forward PE': info.get('forwardPE', 0),
            'PB Ratio': info.get('priceToBook', 0),
            'PS Ratio': info.get('priceToSalesTrailing12Months', 0),
            'EPS': info.get('trailingEps', 0),
            'Book Value': info.get('bookValue', 0),
            'Debt/Equity': info.get('debtToEquity', 0),
            'Current Ratio': info.get('currentRatio', 0),
            'ROE': info.get('returnOnEquity', 0),
            'Dividend Yield': info.get('dividendYield', 0),
            'Profit Margin': info.get('profitMargins', 0),
        }
        
        return data
    except Exception as e:
        return None

# Test with a value stock
value_stocks = ['BRK-B', 'JPM', 'JNJ', 'PG', 'KO']
growth_stocks = ['NVDA', 'TSLA', 'AMZN', 'GOOGL', 'META']

print("Fetching fundamental data...")
all_data = []
for ticker in value_stocks + growth_stocks:
    data = get_fundamentals(ticker)
    if data:
        all_data.append(data)
        
fundamentals = pd.DataFrame(all_data)
print(f"Loaded data for {len(fundamentals)} stocks")

In [ ]:
def calculate_graham_number(eps, book_value):
    """Calculate Graham Number - maximum price to pay."""
    if eps <= 0 or book_value <= 0:
        return 0
    return np.sqrt(22.5 * eps * book_value)

def value_screen(df):
    """Apply Graham-style value screening."""
    df = df.copy()
    
    # Calculate Graham Number
    df['Graham Number'] = df.apply(
        lambda x: calculate_graham_number(x['EPS'], x['Book Value']), axis=1
    )
    
    # Calculate Margin of Safety
    df['Margin of Safety %'] = ((df['Graham Number'] - df['Price']) / df['Graham Number'] * 100).round(1)
    df['Margin of Safety %'] = df['Margin of Safety %'].clip(lower=-100, upper=100)
    
    # Graham Criteria Scoring
    df['PE_Pass'] = (df['PE Ratio'] > 0) & (df['PE Ratio'] < 15)
    df['PB_Pass'] = (df['PB Ratio'] > 0) & (df['PB Ratio'] < 1.5)
    df['Combined_Pass'] = (df['PE Ratio'] * df['PB Ratio'] < 22.5) & (df['PE Ratio'] > 0) & (df['PB Ratio'] > 0)
    df['DE_Pass'] = df['Debt/Equity'] < 100  # D/E < 1.0
    df['CR_Pass'] = df['Current Ratio'] > 2.0
    
    # Value Score (0-5)
    df['Value Score'] = df['PE_Pass'].astype(int) + df['PB_Pass'].astype(int) + \
                        df['Combined_Pass'].astype(int) + df['DE_Pass'].astype(int) + \
                        df['CR_Pass'].astype(int)
    
    return df

# Apply value screen
screened = value_screen(fundamentals)

print(f"\n{'='*80}")
print("GRAHAM-STYLE VALUE SCREEN")
print(f"{'='*80}\n")

display_cols = ['Ticker', 'Price', 'PE Ratio', 'PB Ratio', 'Debt/Equity', 
                'Graham Number', 'Margin of Safety %', 'Value Score']

print(screened[display_cols].sort_values('Value Score', ascending=False).round(2).to_string(index=False))

In [ ]:
def detailed_value_analysis(ticker):
    """Perform detailed value analysis on a single stock."""
    data = get_fundamentals(ticker)
    if not data:
        print(f"Could not get data for {ticker}")
        return
    
    print(f"\n{'='*60}")
    print(f"VALUE ANALYSIS: {data['Name']} ({ticker})")
    print(f"{'='*60}")
    
    print(f"\nCURRENT PRICE: ${data['Price']:.2f}")
    print(f"Market Cap: ${data['Market Cap']/1e9:.1f}B")
    print(f"Sector: {data['Sector']}")
    
    # Graham Number
    graham = calculate_graham_number(data['EPS'], data['Book Value'])
    mos = (graham - data['Price']) / graham * 100 if graham > 0 else 0
    
    print(f"\n{'-'*60}")
    print("GRAHAM CRITERIA CHECK:")
    print(f"{'-'*60}")
    
    # P/E Check
    pe = data['PE Ratio'] if data['PE Ratio'] else 0
    pe_status = 'PASS' if 0 < pe < 15 else 'FAIL'
    print(f"\nP/E Ratio: {pe:.1f} (Graham: <15) [{pe_status}]")
    
    # P/B Check
    pb = data['PB Ratio'] if data['PB Ratio'] else 0
    pb_status = 'PASS' if 0 < pb < 1.5 else 'FAIL'
    print(f"P/B Ratio: {pb:.2f} (Graham: <1.5) [{pb_status}]")
    
    # Combined Check
    combined = pe * pb if pe > 0 and pb > 0 else 0
    combined_status = 'PASS' if 0 < combined < 22.5 else 'FAIL'
    print(f"P/E × P/B: {combined:.1f} (Graham: <22.5) [{combined_status}]")
    
    # D/E Check
    de = data['Debt/Equity'] / 100 if data['Debt/Equity'] else 0
    de_status = 'PASS' if de < 1.0 else 'FAIL'
    print(f"Debt/Equity: {de:.2f} (Graham: <1.0) [{de_status}]")
    
    # Current Ratio Check
    cr = data['Current Ratio'] if data['Current Ratio'] else 0
    cr_status = 'PASS' if cr > 2.0 else 'FAIL'
    print(f"Current Ratio: {cr:.2f} (Graham: >2.0) [{cr_status}]")
    
    # Graham Number and Margin of Safety
    print(f"\n{'-'*60}")
    print("VALUATION:")
    print(f"{'-'*60}")
    print(f"\nEPS: ${data['EPS']:.2f}")
    print(f"Book Value: ${data['Book Value']:.2f}")
    print(f"Graham Number: ${graham:.2f}")
    print(f"Current Price: ${data['Price']:.2f}")
    print(f"Margin of Safety: {mos:.1f}%")
    
    # Verdict
    print(f"\n{'-'*60}")
    print("VERDICT:")
    print(f"{'-'*60}")
    
    if mos > 25 and pe_status == 'PASS' and pb_status == 'PASS':
        print("\nSTRONG VALUE CANDIDATE - Meets Graham criteria with margin of safety")
    elif mos > 0 and combined_status == 'PASS':
        print("\nMODERATE VALUE - Trading below Graham Number")
    elif mos < -25:
        print("\nOVERVALUED - Trading significantly above intrinsic value")
    else:
        print("\nFAIRLY VALUED - Not a clear value opportunity")

# Analyze a classic value stock
detailed_value_analysis('JPM')

In [ ]:
def compare_value_vs_growth(screened_df):
    """Visualize value vs growth characteristics."""
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # P/E Comparison
    ax1 = axes[0, 0]
    colors = ['green' if x < 15 else 'red' for x in screened_df['PE Ratio']]
    bars = ax1.bar(screened_df['Ticker'], screened_df['PE Ratio'], color=colors, alpha=0.7)
    ax1.axhline(y=15, color='black', linestyle='--', label='Graham Max (15)')
    ax1.set_title('P/E Ratio Comparison')
    ax1.set_ylabel('P/E Ratio')
    ax1.tick_params(axis='x', rotation=45)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # P/B Comparison
    ax2 = axes[0, 1]
    colors = ['green' if x < 1.5 else 'red' for x in screened_df['PB Ratio']]
    ax2.bar(screened_df['Ticker'], screened_df['PB Ratio'], color=colors, alpha=0.7)
    ax2.axhline(y=1.5, color='black', linestyle='--', label='Graham Max (1.5)')
    ax2.set_title('P/B Ratio Comparison')
    ax2.set_ylabel('P/B Ratio')
    ax2.tick_params(axis='x', rotation=45)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Margin of Safety
    ax3 = axes[1, 0]
    colors = ['green' if x > 0 else 'red' for x in screened_df['Margin of Safety %']]
    ax3.bar(screened_df['Ticker'], screened_df['Margin of Safety %'], color=colors, alpha=0.7)
    ax3.axhline(y=0, color='black', linestyle='-')
    ax3.axhline(y=25, color='green', linestyle='--', label='Min MoS (25%)')
    ax3.set_title('Margin of Safety')
    ax3.set_ylabel('Margin of Safety %')
    ax3.tick_params(axis='x', rotation=45)
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Value Score
    ax4 = axes[1, 1]
    colors = ['darkgreen' if x >= 4 else ('green' if x >= 3 else ('yellow' if x >= 2 else 'red'))
              for x in screened_df['Value Score']]
    ax4.bar(screened_df['Ticker'], screened_df['Value Score'], color=colors, alpha=0.7)
    ax4.axhline(y=4, color='green', linestyle='--', label='Strong Value (4+)')
    ax4.set_title('Graham Value Score (0-5)')
    ax4.set_ylabel('Value Score')
    ax4.tick_params(axis='x', rotation=45)
    ax4.set_ylim(0, 5.5)
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

compare_value_vs_growth(screened)

---

## Quiz

In [ ]:
quiz = [
    {"q": "Who is known as the 'Father of Value Investing'?", "a": "B", 
     "opts": ["A) Warren Buffett", "B) Benjamin Graham", "C) Charlie Munger", "D) Peter Lynch"]},
    {"q": "What is the margin of safety?", "a": "C",
     "opts": ["A) Stop loss level", "B) Position size limit", 
              "C) Difference between intrinsic value and purchase price", "D) Maximum portfolio allocation"]},
    {"q": "What was Graham's P/E ratio guideline?", "a": "A",
     "opts": ["A) P/E < 15", "B) P/E < 25", 
              "C) P/E < 10", "D) P/E < 5"]},
    {"q": "The Graham Number combines which two metrics?", "a": "B",
     "opts": ["A) P/E and Revenue", "B) EPS and Book Value", 
              "C) Price and Volume", "D) ROE and Debt"]},
    {"q": "What is a typical minimum margin of safety?", "a": "C",
     "opts": ["A) 5-10%", "B) 10-15%", 
              "C) 25-33%", "D) 50%+"]}
]

def run_quiz():
    for i, q in enumerate(quiz):
        print(f"\nQ{i+1}: {q['q']}")
        for opt in q['opts']:
            print(f"   {opt}")
    print("\n" + "="*50)
    print("ANSWERS: 1-B, 2-C, 3-A, 4-B, 5-C")
    
run_quiz()

---

## Summary

- **Value investing** focuses on buying stocks below intrinsic value
- **Margin of safety** protects against errors and provides upside
- **Graham's criteria**: P/E < 15, P/B < 1.5, P/E × P/B < 22.5
- **Patience** is the value investor's advantage
- **Quality matters**: Buffett evolved Graham's quantitative approach

**Tomorrow**: Day 12 - Fundamental Analysis Basics